In [ ]:
filenames = ["<filenames>"]
input_dir = "<input_dir>"
output_dir = "<output_dir>"

In [ ]:
import os
import tarfile

def extract_tar(tar_path, extract_path):
    # Ensure the extract path exists
    os.makedirs(extract_path, exist_ok=True)

    # Open and extract the tar file
    with tarfile.open(tar_path, 'r') as tar:
        tar.extractall(path=extract_path)
    return tar_path  # Returning the tar file path to update the progress bar

In [ ]:
for f in filenames:
    # Construct the full path to the .map.gz file
    mat_file = os.path.join(input_dir, f + '.tar.gz')
    print(f"Processing {mat_file}")
    
    # Check if the file exists before attempting to extract it
    if os.path.isfile(mat_file):
        # Extract the .map.gz file to its .map counterpart in the current directory
        print(f"Extracting {mat_file}")
        extract_tar(mat_file, os.path.join(output_dir, f))
    else:
        print(f"File not found: {mat_file}")

Processing C:\Users\taylo\OneDrive - Queen Mary, University of London\Documents\BOBSL\optical_flow\5093521546186871869.tar.gz
Extracting C:\Users\taylo\OneDrive - Queen Mary, University of London\Documents\BOBSL\optical_flow\5093521546186871869.tar.gz


: 

In [ ]:
import os
import glob
import numpy as np
import cv2  # pip install opencv-python
import torch  # pip install torch
from tqdm import tqdm  # pip install tqdm

def load_flow_sequence(
    folder_path: str,
    pattern: str = '*.jpg',
    as_tensor: bool = True,
) -> np.ndarray or torch.Tensor:

    files = sorted(glob.glob(os.path.join(folder_path, pattern)))
    if not files:
        raise FileNotFoundError(f"No files match {pattern} in {folder_path}")

    decoded_flows = []

    for fpath in tqdm(files, desc='Decoding flow frames', unit='frame'):

        bgr = cv2.imread(fpath, cv2.IMREAD_COLOR)
        if bgr is None:
            raise IOError(f"Could not read image {fpath}")
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

        hsv = cv2.cvtColor((rgb * 255).astype(np.uint8), cv2.COLOR_RGB2HSV).astype(np.float32)
        H = hsv[..., 0]           # Hue channel [0,180)
        S = hsv[..., 1] / 255.0   # Saturation [0,1]

        theta = H / 180.0 * 2.0 * np.pi
        u = S * np.cos(theta)
        v = S * np.sin(theta)

        flow = np.stack([u, v], axis=-1).astype(np.float32)  # (H, W, 2)
        decoded_flows.append(flow)

    flows_np = np.stack(decoded_flows, axis=0)  # (T, H, W, 2)

    if as_tensor:
        return torch.from_numpy(flows_np).permute(0, 3, 1, 2)
    else:
        return flows_np


In [8]:
for f in filenames:
    # Construct the full path to the .map.gz file
    path = os.path.join(dir, f)
    
    # Check if the file exists before attempting to extract it
    if os.path.isdir(path):
        print(f"Processing {path}")
        flows = load_flow_sequence(path, pattern="*.jpg", as_tensor=True)
        print("Loaded flow tensor with shape:", flows.shape)
        torch.save(flows, os.path.join(output_dir, f + '_optical_flow_tensor.pt'))

    else:
        print(f"File not found: {path}")

Processing C:\Users\taylo\OneDrive\Documents\Dissertation_Work\BOBSL\optical_flow_data\5090916219025116054


Decoding flow frames:  68%|██████▊   | 59451/87333 [30:14<14:10, 32.77frame/s]  


MemoryError: Unable to allocate 768. KiB for an array with shape (256, 256, 3) and data type float32